In [1]:
import sys
sys.path.append('../')

from src import data_processing
import numpy as np

# Tiền xử lí dữ liệu

Lựa chọn các đặc trưng cần thiết, ảnh hưởng lớn đến việc xác định khả năng khách hàng rời đi.  
Qua quá trình phân tích visualization và thử nghiệm (thử và sai), ta quyết định chọn các đặc trưng là:  
- `Attrition_Flag`,
-  `Total_Trans_Ct`,
-  `Total_Trans_Amt`,
-  `Total_Ct_Chng_Q4_Q1`,
-  `Total_Revolving_Bal`,
-  `Avg_Utilization_Ratio`,
-  `Total_Relationship_Count`,
-  `Months_Inactive_12_mon`,
-  `Credit_Limit`

Việc lựa chọn các đặc trưng giúp mô hình tập trung vào những dữ liệu quan trọng, tránh bị nhiễu hay ảnh hưởng từ các đặc trưng kém quan trọng.

In [2]:
raw_data = np.genfromtxt('../data/raw/BankChurners.csv', delimiter=',', encoding='utf8', dtype=str)
raw_data = np.array([[cell.replace('"', '') for cell in row] for row in raw_data])
header = list(raw_data[0])
data = raw_data[1:]
selected_features = [
    'Attrition_Flag',
    'Total_Trans_Ct',
    'Total_Trans_Amt',
    'Total_Ct_Chng_Q4_Q1',
    'Total_Revolving_Bal',
    'Avg_Utilization_Ratio',
    'Total_Relationship_Count',
    'Months_Inactive_12_mon',
    'Credit_Limit'
]

selected_indices = [header.index(col) for col in selected_features]

data_selected = data[:, selected_indices]
header_selected = selected_features

print("Selected data size:", data_selected.shape)

Selected data size: (10127, 9)


Mã hóa cột `Attrition_Flag` để huấn luyện mô hình phân loại nhị phân với:
- `1`: Attrited Customer
- `0`: Existing Customer

In [3]:
processed_data = data_selected.copy()
header = header_selected.copy()

target_idx = header.index('Attrition_Flag')
attrition_column = processed_data[:, target_idx]

target_encoded = np.where(attrition_column == 'Attrited Customer', 1, 0)
processed_data[:, target_idx] = target_encoded

Thêm đặc trưng mới là `Custom_Utilization_Ratio` từ `Total_Revolving_Bal` và `Credit_Limit` theo công thức:  
```
Utilization_Ratio = Total_Revolving_Bal / Credit_Limit
```
**Ý nghĩa**: Tỷ lệ sử dụng tín dụng là một trong những chỉ số quan trọng nhất để đánh giá sức khỏe tài chính và rủi ro của khách hàng. Một tỷ lệ cao có thể cho thấy khách hàng đang gặp khó khăn về tài chính, đây là một yếu tố dự báo mạnh mẽ về khả năng rời bỏ. Đặc trưng này chứa đựng thông tin tổng hợp tốt hơn nhiều so với việc chỉ dùng `Total_Revolving_Bal` hoặc `Credit_Limit` một cách riêng lẻ.

In [4]:
processed_data = processed_data.astype(float)

revolving_bal_idx = header.index('Total_Revolving_Bal')
credit_limit_idx = header.index('Credit_Limit')

revolving_bal = processed_data[:, revolving_bal_idx]
credit_limit = processed_data[:, credit_limit_idx]
utilization_ratio = revolving_bal / (credit_limit + 1e-6) 

processed_data = np.c_[processed_data, utilization_ratio]
header.append('Custom_Utilization_Ratio')

print("Add new feature Custom_Utilization_Ratio.")
print("New shape:", processed_data.shape)

Add new feature Custom_Utilization_Ratio.
New shape: (10127, 10)


## Xử lí outliners và chuẩn hóa:
1. **Xử lí outliners.**
- Phương pháp: Chúng ta đã sử dụng phương pháp IQR để xác định các giá trị ngoại lệ.  
Một giá trị được coi là ngoại lệ nếu nó nằm ngoài khoảng [Q1 - 1.5*IQR, Q3 + 1.5*IQR].
- Cách thực hiện:
  - Áp dụng phương pháp Loại bỏ.
  - Quy trình lặp qua các cột đặc trưng quan trọng đã chọn (`Credit_Limit, Total_Trans_Amt, v.v.`).
  - Đối với mỗi cột, các chỉ số của những hàng chứa giá trị ngoại lệ đã được xác định.
  - Tất cả các chỉ số này được tổng hợp lại và các hàng tương ứng đã bị xóa hoàn toàn khỏi bộ dữ liệu.
2. **Chuẩn hóa.**
- Phương pháp: Chúng ta đã sử dụng phương pháp Standardization (Z-score Scaling).
- Hành động:
  - Mỗi giá trị trong các cột đặc trưng đã được biến đổi theo công thức:  
`Z = (giá_trị - trung_bình) / độ_lệch_chuẩn`

In [5]:
target_column = processed_data[:, 0].reshape(-1, 1)
feature_columns = processed_data[:, 1:]
feature_header = header[1:]

print("Remove Outliers")

outlier_check_cols = [
    'Credit_Limit', 'Total_Revolving_Bal', 
    'Total_Trans_Amt', 'Total_Trans_Ct', 
    'Custom_Utilization_Ratio'
]

indices_to_remove = set()

for col_name in outlier_check_cols:
    if col_name in feature_header:
        col_idx = feature_header.index(col_name)
        column_data = feature_columns[:, col_idx]

        outlier_indices = data_processing.get_outlier_indices_iqr(column_data)
        
        print(f"[{col_name}] Finds {len(outlier_indices)} outliers.")

        indices_to_remove.update(outlier_indices)
    else:
        print(f"'{col_name}' is not in header.")

indices_to_remove = sorted(list(indices_to_remove))

print(f"\nTotal removed outliners: {len(indices_to_remove)}")

original_rows = feature_columns.shape[0]

if len(indices_to_remove) > 0:
    feature_columns = np.delete(feature_columns, indices_to_remove, axis=0)
    target_column = np.delete(target_column, indices_to_remove, axis=0)

print(f"Data shape after removing outliers: {feature_columns.shape[0]} row")
print(f"Removed: {original_rows - feature_columns.shape[0]} row")

# Normalization
standardized_features = data_processing.standardize_features(feature_columns)

final_processed_data = np.hstack((target_column, standardized_features))

final_header = [header[0]] + feature_header

Remove Outliers
[Credit_Limit] Finds 984 outliers.
[Total_Revolving_Bal] Finds 0 outliers.
[Total_Trans_Amt] Finds 896 outliers.
[Total_Trans_Ct] Finds 2 outliers.
[Custom_Utilization_Ratio] Finds 0 outliers.

Total removed outliners: 1684
Data shape after removing outliers: 8443 row
Removed: 1684 row


## Lưu dữ liệu đã qua xử lí.

In [7]:
output_path = '../data/processed/data_processed.csv'

data_to_save = np.vstack((header, final_processed_data.astype(str)))

np.savetxt(output_path, data_to_save, delimiter=',', fmt='%s', encoding='utf-8')

print(f"Save successfully: {output_path}")

Save successfully: ../data/processed/data_processed.csv
